# Advent of Code 2021 (PowerShell)

## Day 1 - Sonar Sweep

[Puzzle Link](https://adventofcode.com/2021/day/1)

### Input

In [ ]:
[int[]]$Readings = Get-Content -Path .\PuzzleInput\day1.txt

#### Learning Points

- By default, `Get-Content` will read the file line by line, creating an array of (string) lines.
- You need to compare the current and previous reading as numbers and not as strings.
Make PowerShell do the work by *casting* the variable as an array of integers, `[int[]]`.

### Puzzle 1

Using input file, count the number of times a depth measurement increases.

In [ ]:
# Direct variable assignment
$Increased = for ($Count = 1; $Count -le $Readings.Count; $Count++) {
    if ($Readings[$Count] -gt $Readings[$Count-1]) { 1 }
}
$Increased.Count

1451


In [ ]:
# Incrementing variable
$Increased = 0
for ($Count = 1; $Count -le $Readings.Count; $Count++) {
    if ($Readings[$Count] -gt $Readings[$Count-1]) { $Increased++ }
}
$Increased

1451


#### Learning Points

- You will need to use a loop to cycle through each of the readings.
I've chosen to use a `for` loop which allows me to initialize a counter, define the condition that ends the loop, and increment the counter.
- You can increment or decrement an integer by using `++` or `--`.
- Instead of defining a variable that would contain the number of times the readings increased, I wanted to show you that you can assign the results of a `for` loop,
and many other loops and expressions, directly to a variable.
By doing so, I can output `1` and because of the variable assignment, `$Increased = for (...) { ... }`, it becomes an array of `1's`.
To get the answer, I use the array type property, `Count`.
- I've also included a second method which will likely take less memory and complete faster.
This method uses a simple array indexing to refer to a specific reading, `$Readings[$Count]`.
Note that you can also use a mathematical expression.

### Puzzle 2

Count the number of times the sum of measurements in this sliding window increases from the previous sum.

In [ ]:
$GroupCount = 0
$GroupIncreased = 0
do {
    $PreviousGroupSum = ($Readings[$GroupCount..($GroupCount+2)] | Measure-Object -Sum).Sum
    $CurrentGroupSum = ($Readings[($GroupCount+1)..($GroupCount+3)] | Measure-Object -Sum).Sum
    if ($CurrentGroupSum -gt $PreviousGroupSum) {
        $GroupIncreased++
    }
    $GroupCount++
} while ($GroupCount -lt $Readings.Count)
$GroupIncreased

1395


#### Learning Points

- Again we will need a loop and I've chosen the `do {...} while (...)` loop.
The `do` portion of this loop accepts a list of statements to execute and the `while` portion defines the condition.
When the condition is no longer met, the loop ends. 
- There's a lot going on with lines 4 and 5.
The `$Readings` variable still contains the input from the first puzzle.
Since we need to group 3 consecutive readings, I'm using a range index of the `$Readings` array.
This is a bit more advanced in that I specify the starting and ending index number which returns that portion of the array.
For example, if `$GroupCount = 30` then line 4 would mean `$Readings[30..32]`, or give me the 29th, 30th, and 31st item in the array.
Many programming and scripting languages uses a zero index, meaning that the first item is at index 0 or position 0.
- Once we have the array of 3 readings, we use `Measure-Object -Sum` to sum the three together.
- We then access the `Sum` property using dot notation.
You can access any property of an object, in this case the object returned by the `Measure-Object` command which is of type `Microsoft.PowerShell.Commands.GenericMeasureInfo`.
Pipe any object into `Get-Member` to see the object's type and properties.

## Day 2 - Dive!

[Puzzle Link](https://adventofcode.com/2021/day/2)

### Input

In [ ]:
$SubMoves = Get-Content -Path .\PuzzleInput\day2.txt

### Puzzle 1

What do you get if you multiply your final horizontal position by your final depth?

In [ ]:
$Position = 0
$Depth = 0
foreach ($Move in $SubMoves) {
    $Direction,[int]$Distance = $Move.Split(' ')
    switch ($Direction) {
        'forward'   { $Position += $Distance }
        'down'      { $Depth += $Distance }
        'up'        { $Depth -= $Distance }
    }
}
'Horizontal Position: {0}, Depth {1}, Location: {2}' -f $Position,$Depth,($Position * $Depth)

Horizontal Position: 1991, Depth 911, Location: 1813801


#### Learning Points

- Again, I selected another type of loop.
Since each move does not rely on others, the `foreach` loop allows us to simply iterate through the array of moves.
It will stop when it gets to the end of the list.
- Each move, or line in the input file, consists of a direction and a distance.
We need to break these up and we can do that by using the `.Split()` method of strings.
- And since we know we are going to be using the distance in mathematical expressions, it's best to cast the number as an integer.
- PowerShell allows you too assign multiple variables using a single command.
Line 4 shows that we want to assign the first piece of the move as `$Direction` and the second piece as `[int]$Distance`.
- Next, we need to perform some arithmetic based on the move's direction.
We have a possible 3 directions, so a `switch` statement works great here.
Given an argument to evaluate, you can specify any number of conditions based on that argument.
For each condition, we perform the required math operation.
- Instead of providing the raw answer, I wanted to make the output a little more meaningful.
For quite some time now, I've defaulted to using the .NET string formatting statement, `-f`.
With it, you provide your text in double or single quotes (I prefer single as I can include doubles as strings inside if necessary) with place holders `{#}`.
As you can see, the number (#) is zero-indexed.
The number of placeholders must match the number of elements on the right-side of the `-f` in the statement.

### Puzzle 2

With updated instructions, what do you get if you multiply your final horizontal position by your final depth?

In [ ]:
$Position = 0
$Depth = 0
$Aim = 0
foreach ($Move in $SubMoves) {
    $Direction,[int]$Distance = $Move.Split(' ')
    switch ($Direction) {
        'forward'   { 
            $Position += $Distance
            $Depth += $Distance * $Aim
        }
        'down'      { $Aim += $Distance }
        'up'        { $Aim -= $Distance }
    }
}
'Horizontal Position: {0}, Depth {1}, Location: {2}' -f $Position,$Depth,($Position * $Depth)

Horizontal Position: 1991, Depth 984716, Location: 1960569556


#### Learning Points

- Most of our solution for the first puzzle of day 2 has been used for this puzzle.
This is a small but significant concept.
Your PowerShell code should be written such that it can easily be reused with or without slight modification.
For more complicated requirements, this means writing an advanced function that performs a single thing well.
- For the updated requirements, we add the `$Aim` variable and update the `switch` actions/math operations.
- In the previous learning point, I failed to mention the `+=` and `-=` assignment operators.
Each one adds or subtracts, respectively, the variable by the amount of the value on the right.
`Get-Help assignment_operators` will give you the full list.

## Day 3